<a href="https://colab.research.google.com/github/NgSehBin/2526-workshop/blob/PEKOM/AI_Agents_in_Action_Hands_On.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI Agents in Action Workshop: Hands-on Session**

# **AI Newsletter Agents**

## Agent 1: Research Agent 🕵️‍♂️
**Role:**
Search the internet to find the most relevant AI news.

**Tools:** Web Search Tool

**What it does:**
1. Receives a specific date or topic
2. Searches the web for the top AI/Tech story of that day
3. Filters for the most significant development and collects sources
4. Summarize the contents before sending to next agent


## Agent 2: Writer Agent ✍️
**Role:**
Takes the summarized report and transforms it into engaging content.

**What it does:**
1. Reads the raw research from Agent 1.
2. Writes a catchy headline and a fun summary.
3. Writes a detailed "Image Prompt" describing a funny meme visual based on the news.




## Agent 3: Thumbnail Agent 🎨
**Role:**
The creative visualizer who turns text into pixels.

**Tools:**
- **Custom HuggingFace Tool**

**What it does:**
1. Reads the "Image Prompt" created by the Writer.
2. Sends the prompt to the image generation model.
3. Saves the final meme to your file system.

### **Step 1: Setting Up!**

In [ ]:
!pip install "crewai[tools]"
!pip install tavily-python
!pip install groq litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 kB 31.7

In [ ]:
import os
from google.colab import userdata

hf_key = userdata.get('')
tavily_api_key = userdata.get("")
groq_api_key = userdata.get("")

# OR directly paste your API keys here, but make sure to not share it to others


### **Step 2: Initialize LLM**

Before creating agents, we must create the "brain" that powers them. In CrewAI, agents rely on a Large Language Model (LLM). Agents will use OpenAI models by default
<br>

**Important Note:** Not all models have the tool calling capabilities, so make sure you check before getting frustrated on why your agent is not working

In [ ]:
# We are using 2 models offered by Groq
# Model ID: "groq/Llama-3.1-8B-instant" and "groq/llama-3.3-70b-versatile"
# Backup Models : qwen/qwen3-32b, meta-llama/llama-4-maverick-17b-128e-instruct



<details>
    <summary>Show Full Code</summary>

```python
from crewai import LLM

llm = LLM(
    model="groq/Llama-3.1-8B-instant",
    api_key=groq_api_key,

)

llm2 = LLM(
    model="groq/llama-3.3-70b-versatile",
    api_key=groq_api_key
)
```
</details>

### **Step 3:** Define Tools and Schemas

**1. Schemas:**

AI models normally talk in "paragraphs," and have multiple formatting of output. which can be messy if you are trying to pass to tools. Schema forces agent to give a predefined structure of output (mostly in json)

We create a `NewsArticle` class to tell the Writer agent exactly how to format the output. It *must* return a Headline, Summary, and Image Prompt.


<details>
    <summary>Show Full Code</summary>

```python
from pydantic import BaseModel, Field

class NewsArticle(BaseModel):
    headline: str = Field(description="Funny news headlines")
    summary: str = Field(description="Rewritten summary of the news in a funny way")
    image_prompt: str = Field("Prompt to be sent to image generation model")

class ImagePromptInput(BaseModel):
    prompt: str = Field(description="The image generation prompt")

```
</details>

**2. Using CrewAI Built in Tools**

**TavilySearchTool:** Gives the agent access to real-time internet data.

Tool Paremeters:
*   `max_results`: Limits how many links the agent reads (keeps it fast).


In [ ]:
# Importing crewAI web search tools


<details>
    <summary>Show Full Code</summary>

```python
from crewai_tools import TavilySearchTool

tavily_tool = TavilySearchTool(
    max_results=3,
    api_key=tavily_api_key
)
```
</details>

**3. Build our own Custom Tools**

You can also customize agents with your own custom tools

**HuggingFaceImageGenTool:** A custom tool we build to connect to an image generator model (`Stable Diffusion`) using HuggingFace Inference.


In [ ]:
from huggingface_hub import InferenceClient
from typing import Type
from io import BytesIO
import requests
from PIL import Image

#Import CrewAI module


# == Tool Info ==
# Name: HuggingFace Image Generator
# Desc: Generates an image based on a text prompt. seful for visualizing concepts. The input must be a specific image description.
# Image Model : stabilityai/stable-diffusion-xl-base-1.0


<details>
    <summary>Show Full Code</summary>

```python
from huggingface_hub import InferenceClient
from typing import Type
from io import BytesIO
import requests
from PIL import Image

from crewai.tools import BaseTool


class HuggingFaceImageGenTool(BaseTool):
    name: str = "HuggingFace Image Generator"
    description: str = (
        "Generates an image based on a text prompt. "
        "Useful for visualizing concepts. "
        "The input must be a specific image description."
    )
    args_schema: Type[BaseModel] = ImagePromptInput
    def _run(self, prompt: str) -> str:
        try:
            client = InferenceClient(
                provider="hf-inference",
                api_key=hf_key,
            )

            print(prompt)

            model_id = "stabilityai/stable-diffusion-xl-base-1.0"

            image = client.text_to_image(
                prompt,
                model=model_id,
            )

            output_filepath = "hf_generated_image.png"
            image.save(output_filepath)

            return f"Image successfully generated and saved to '{output_filepath}' based on prompt: '{prompt}'"

        except Exception as e:
            return f"Error generating image: {str(e)}"

hf_image_gen_tool = HuggingFaceImageGenTool()
```
</details>

### **Step 4: Build Agents**

Now we create our team members. Remember, an agent is like an employee with a specific job description.



**Parameters for Agents:**

*   **`role`** Defines the agent's function within the crew.

*   **`goal`** Tells the agent exactly what it needs to achieve to finish the task

*   **`backstory`:**  A detailed description of the agent's history, expertise, and attitude. Good backstory = better personalized response
*   **`tools`**: (Optional) The specific tools given to this agent
*   **`verbose`**: (Optional) If set to `True`, we can see how the thought process of an agent
*   **`memory`**:(Optional) Allows the agent to remember past interactions to improve context. (Good for chatbot)

In [ ]:
# Import Agent Module


# 1. Research Agent
# Role : AI News Researcher
# Goal: Find the single most important AI topic for a specific dat
# Backstory: You are an expert tech analyst. You browse the web to find the hottest AI news of the day. You focus on quality over quantity.

# 2. Writer Agent
# Role : Tech Newsletter Writer
# Goal: Write a short, catchy news piece and a prompt for a meme image
# Backstory: You are a witty tech writer who makes complex AI news sound fun and simple. You also know how to describe funny images.

# 3. Thumbnail Agent
# Role : AI Meme Artist
# Goal: Generate an image based on a promp
# Backstory: You are a creative AI artist. You take text descriptions and turn them into visual reality using generative tools.


<details>
    <summary>Show Full Code</summary>

```python
from crewai import Agent

researcher = Agent(
    role='AI News Researcher',
    goal='Find the single most important AI topic for a specific date',
    backstory='You are an expert tech analyst. You browse the web to find the hottest AI news of the day. You focus on quality over quantity.',
    verbose=True,
    memory=True,
    tools=[tavily_tool],
    llm=llm2,
    max_iter = 3,
    respect_context_window = True

)

writer = Agent(
    role='Tech Newsletter Writer',
    goal='Write a short, catchy news piece and a prompt for a meme image',
    backstory='You are a witty tech writer who makes complex AI news sound fun and simple. You also know how to describe funny images.',
    verbose=True,
    memory=True,
    llm=llm,
    respect_context_window = True,
    max_iter = 1,


)

memer = Agent(
    role='AI Meme Artist',
    goal='Generate an image based on a prompt',
    backstory='You are a creative AI artist. You take text descriptions and turn them into visual reality using generative tools.',
    verbose=True,
    memory=True,
    tools=[hf_image_gen_tool],
    llm=llm,
    respect_context_window = True,
    max_iter = 1,
)
```
</details>

### **Step 5:** Create Tasks

Agents need specific assignments to work on. A `Task` connects an Agent to a Goal.

</br>


In [ ]:
from crewai import Task

# Task 1: Find the news
"""
Description:
        Search for AI news specifically for this date: {date}.
        Identify the TOP 1 most significant AI news or story of that day.
        When using the Tavily Search tool, your Action Input MUST be a plain string for the 'query' argument, for example: 'AI news for {date}'. DO NOT wrap the query in a dictionary with 'description' and 'type' keys.
        Collect 2-3 distinct sources/links for this story. "
        Search only ONCE

Expected Output:
        A summary of the top AI news story of the day with sources.
"""


# Task 2: Write the content and image prompt
"""
Description:
        Based on the research provided, create a newsletter entry.
        Your writing must be funny and witty to attract audience
        You MUST format your output as a structured object containing:
        a Headline, a Short Summary, and a detailed Image Prompt.

Expected Output:
        A summary of the top AI news story of the day with sources.
"""


# Task 3: Generate the Image
"""
Description:
        Look at the 'Image Prompt' provided by the Writer Agent's output.
        Use the HuggingFace Image Generator tool to generate an image based on that specific prompt.
        Ensure the image is saved to the file system.

Expected Output:
        Show the writer agent Headline, News content in string format
"""


ModuleNotFoundError: No module named 'crewai'

<details>
    <summary>Show Full Code</summary>

```python
from crewai import Task

research_task = Task(
    description=(
        """Search for AI news specifically for this date: {date}. "
        "Identify the TOP 1 most significant AI news or story of that day. "
        "When using the Tavily Search tool, your Action Input MUST be a plain string for the 'query' argument, for example: 'AI news for {date}'. DO NOT wrap the query in a dictionary with 'description' and 'type' keys. "
        "Collect 2-3 distinct sources/links for this story. "
        "Search only ONCE"""
    ),
    expected_output='A summary of the top AI news story of the day with sources.',
    agent=researcher,
)

write_task = Task(
    description=(
        """Based on the research provided, create a newsletter entry.
        Your writing must be funny and witty to attract audience
        You MUST format your output as a structured object containing:
        a Headline, a Short Summary, and a detailed Image Prompt."""
    ),
    output_pydantic=NewsArticle,
    expected_output='A pydantic object with headline, summary, and image_prompt.',
    agent=writer,
)

meme_task = Task(
    description=(
        """Look at the 'Image Prompt' provided by the Writer Agent's output.
        Use the HuggingFace Image Generator tool to generate an image based on that specific prompt.
        Ensure the image is saved to the file system."""
    ),
    expected_output='Show the writer agent Headline, News content and status for the image in string format',
    agent=memer,
)
```
<details>

**Parameters for Task:**
*   **`description`**: A clear, step-by-step instruction of what needs to be done.
*   **`expected_output`**: A definition of what the final result should look like (e.g., "A specific summary").
*   **`agent`**: Assigns the task to a specific agent.
*   **`output_pydantic`**: (Used in the Write Task) Forces the agent to output the data in the strict JSON format we defined earlier (NewsArticle).

### **Step 6:** Build a Crew!

The `Crew` object brings your Agents and Tasks together into a workflow.


**Parameters for Crew:**
*   **`agents`**: The list of all agents in the team.
*   **`tasks`**: The list of tasks to be executed.
*   **`process`**: How the crew works. `Process.sequential` means they work one after another (Task 1 -> Task 2 -> Task 3).
*   **`planning`**: If `True`, the Crew will take a moment before starting to plan the most efficient way to tackle the tasks.

<details>
    <summary>Show Full Code</summary>

```python
from crewai import Crew, Process

crew = Crew(
    agents=[researcher, writer, memer],
    tasks=[research_task, write_task, meme_task],
    process=Process.sequential,
    verbose=True,
)**bold text**
```
<details>

### **Final Step:** Kickoff!

We run the crew. We pass the `date` input into the tasks, and the agents will start working immediately.


Enter the date for the news (e.g., 'September 12, 2024'): February 19 2004

Starting CrewAI for date: February 19 2004...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8f8f44a9-25d3-4df3-88cc-308e4b2a93b2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI News Researcher                                                                                      │
│                                                                                                                 │
│  Task: Search for AI news specifically for this date: February 19 2004. "                                       │
│          "Identify the TOP 1 most significant AI news or story of that day. "                                   │
│          "When using the Tavily Search tool, your Action Input MUST be a plain string for the 'query'           │
│  argument, for example: 'AI news for February 19 2004'. DO NOT wrap the query in a dictionary with              │
│  'description' and 'type' keys. "                                                                               │
│          "Collect 2-3 distinct sources/links for this story. "                                                  │
│          "Search only ONCE                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI Meme Artist                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│   I'm depending on you, this is the moment of truth!                                                            │
│                                                                                                                 │
│  {                                                                                                              │
│      "headline": "Fowl Play: Avian Influenza Takes its Revenge on Canadian Fowl",                               │
│      "summary": "Get ready for a shocking tale of disease, fowl play, and utter chaos in the poultry world. It  │
│  turns out that the top AI news story for February 19, 2004, is about a nasty Avian Influenza outbreak in       │
│  British Columbia, Canada. Yes, you read that right - it's Avian Influenza, not Artificial Intelligence,        │
│  that's behind this poultry pandemonium! According to reports, officials confirmed the presence of AI on this   │
│  fateful day, and before you could say 'fowl' twice, over 365,000 chickens and turkeys found themselves in a    │
│  precarious situation. Sounds like a real hoot, doesn't it?",                                                   │
│      "image_prompt": "A cartoon illustration of a frazzled farmer in a biohazard suit, standing among a group   │
│  of frantic chickens and turkeys. The farmer's face is a mix of horror and despair, with a speech bubble        │
│  reading 'AI Apocalypse!' above their head. In the background, a giant chicken with a flu mask is standing on   │
│  top of a pile of feathers, looking mighty concerned. The image should have a comical, exaggerated tone and     │
│  bright, cartoon-like colors."                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 179773ef-56e7-4e96-a0c6-71ef95a7a086                                                                     │
│  Agent: AI Meme Artist                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()



########################
## NEWSLETTER RESULT ##
########################

 I'm depending on you, this is the moment of truth!
 
{
    "headline": "Fowl Play: Avian Influenza Takes its Revenge on Canadian Fowl",
    "summary": "Get ready for a shocking tale of disease, fowl play, and utter chaos in the poultry world. It turns out that the top AI news story for February 19, 2004, is about a nasty Avian Influenza outbreak in British Columbia, Canada. Yes, you read that right - it's Avian Influenza, not Artificial Intelligence, that's behind this poultry pandemonium! According to reports, officials confirmed the presence of AI on this fateful day, and before you could say 'fowl' twice, over 365,000 chickens and turkeys found themselves in a precarious situation. Sounds like a real hoot, doesn't it?",
    "image_prompt": "A cartoon illustration of a frazzled farmer in a biohazard suit, standing among a group of frantic chickens and turkeys. The farmer's face is a mix of horror and despai

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8f8f44a9-25d3-4df3-88cc-308e4b2a93b2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:  I'm depending on you, this is the moment of truth!                                              │
│                                                                                                                 │
│  {                                                                                                              │
│      "headline": "Fowl Play: Avian Influenza Takes its Revenge on Canadian Fowl",                               │
│      "summary": "Get ready for a shocking tale of disease, fowl play, and utter chaos in the poultry world. It  │
│  turns out that the top AI news story for February 19, 2004, is about a nasty Avian Influenza outbreak in       │
│  British Columbia, Canada. Yes, you read that right - it's Avian Influenza, not Artificial Intelligence,        │
│  that's behind this poultry pandemonium! According to reports, officials confirmed the presence of AI on this   │
│  fateful day, and before you could say 'fowl' twice, over 365,000 chickens and turkeys found themselves in a    │
│  precarious situation. Sounds like a real hoot, doesn't it?",                                                   │
│      "image_prompt": "A cartoon illustration of a frazzled farmer in a biohazard suit, standing among a group   │
│  of frantic chickens and turkeys. The farmer's face is a mix of horror and despair, with a speech bubble        │
│  reading 'AI Apocalypse!' above their head. In the background, a giant chicken with a flu mask is standing on   │
│  top of a pile of feathers, looking mighty concerned. The image should have a comical, exaggerated tone and     │
│  bright, cartoon-like colors."                                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<details>
    <summary>Show Full Code</summary>

```python
from crewai import Crew, Process

date_input = input("Enter the date for the news (e.g., 'September 12, 2024'): ")

print(f"\nStarting CrewAI for date: {date_input}...\n")

crew = Crew(
    agents=[researcher, writer, memer], # Include all agents
    tasks=[research_task, write_task, meme_task], # Include all tasks
    process=Process.sequential,
    verbose=True,
)

result = crew.kickoff(inputs={'date': date_input})

from IPython.display import Image, display, Markdown

print("\n\n########################\n## NEWSLETTER RESULT ##\n########################\n")
print(result)

from google.colab import output
output.clear()
```
</details>

In [ ]:
from IPython.display import Image, display
image_path = "hf_generated_image.png"
display(Image(filename=image_path))

## **Enhanced Version** (Hierarchial Process)

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, Optional
from crewai import Agent, Task, Process, LLM, Crew

# --- LLM CONFIGURATION ---

groq_api_key = userdata.get('GROQ_API')

llm_model = LLM(
    model="groq/llama-3.3-70b-versatile",
    api_key=groq_api_key
)

# ======= TOOLS =======

class EmailToolInput(BaseModel):
    subject: str = Field(description="The subject line of the email")
    body: str = Field(description="The main content/summary of the email")
    receiver_email: str = Field(description="The email address extracted from the user's request.")

class GmailSendingTool(BaseTool):
    name: str = "Gmail Sending Tool"
    description: str = "Sends a text-based email via Gmail."
    args_schema: Type[BaseModel] = EmailToolInput

    def _run(self, subject: str, body: str, receiver_email: str) -> str:
        sender_email = userdata.get('GMAIL_EMAIL')
        sender_password = userdata.get('GMAIL_APP_PASS')

        try:
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = receiver_email
            msg['Subject'] = subject
            msg.attach(MIMEText(body, 'plain'))

            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()
                server.login(sender_email, sender_password)
                server.send_message(msg)
            return f"Email sent successfully to {receiver_email}."
        except Exception as e:
            return f"Error sending email: {str(e)}"

email_tool = GmailSendingTool()

# ======= AGENTS
# 1. THE MANAGER
manager = Agent(
    role="Workflow Director",
    goal="Orchestrate the pipeline: Research -> Writing -> Distribution.",
    backstory="You are the boss. You assign tasks to the Researcher, Writer, and Publisher. You ensure data flows from one to the other.",
    llm=llm_model,
    verbose=True,
    allow_delegation=True,
    memory=True,
)

# 2. RESEARCHER
researcher = Agent(
    role='AI News Researcher',
    goal='Find the single most important AI topic.',
    backstory='You are a fact-finder. You find the news and summarize the facts.',
    verbose=True,
    tools=[tavily_tool],
    llm=llm_model,
)

# 3. WRITER
writer = Agent(
    role='Content Stylist',
    goal='Adapt the content tone ONLY if requested.',
    backstory=(
        "You receive raw research. "
        "Logic:"
        "1. If the user request implies 'funny', 'witty', or 'style', you rewrite the research."
        "2. If the user request is just 'find info', you return the research EXACTLY as is."
    ),
    verbose=True,
    llm=llm_model,
    max_iter=2
)

# 4. PUBLISHER
publisher = Agent(
    role='Distribution Manager',
    goal='Handle email delivery if requested.',
    backstory=(
        "You receive the final content (either styled or raw). "
        "Logic:"
        "1. If the user provided an email address AND wants it sent: Use the Gmail Tool."
        "2. If no email is provided or they say 'no email': Just output the content as text."
    ),
    verbose=True,
    tools=[email_tool],
    llm=llm_model,
    max_iter=1
)

# ======= TASKS =======

# Task 1: Research
research_task = Task(
    description="Search for the information requested in: '{request}'. Return a comprehensive summary.",
    expected_output='A summary of the facts.',
)

# Task 2: Write/Style
class NewsContent(BaseModel):
    headline: str = Field(description="The headline (funny or serious)")
    body: str = Field(description="The body text (funny or serious)")

write_task = Task(
    description=(
        "1. Review the original request: '{request}'.\n"
        "2. Review the Research Context.\n"
        "3. IF the user wants a specific style (funny, witty), rewrite the research.\n"
        "4. IF NOT, simply copy the research into the body field."
    ),
    expected_output='Structured news content.',
    output_pydantic=NewsContent,
    context=[research_task],
)

# Task 3: Publish/Email
email_task = Task(
    description=(
        "1. Review the original request: '{request}'.\n"
        "2. Retrieve the content from the Writer.\n"
        "3. CHECK: Is there a valid email address in the request?\n"
        "   - YES: Send email via Gmail Tool using the headline as Subject.\n"
        "   - NO: Return the string 'Email skipped' followed by the content."
        "4. CRITICAL: This is the final step. Return the result as the Final Answer to the user."

    ),
    expected_output='Final status string.',
    context=[write_task],
)

# ======= CREW =======

crew = Crew(
    agents=[researcher, writer, publisher],
    tasks=[research_task, write_task, email_task],
    process=Process.hierarchical,
    manager_agent=manager,
    verbose=True,
    max_iter=2,
)



In [ ]:
inputs = {'request': 'Find news on 25 nov 2025 and rewrite funny , send to s2191266@siswa.um.edu.my'}
crew.kickoff(inputs=inputs)

Enter your data and preferfrferfences:today, about a.i , pessiypenaldo710@gmail.com


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 16919ac3-7bed-4c96-a16f-80d9153d8578                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chief Editor                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Analyze the original request: 'today, about a.i , pessiypenaldo710@gmail.com'.                         │
│                                                                                                                 │
│          YOUR JOB (As Manager):                                                                                 │
│          1. Check if the user wants an email sent.                                                              │
│             - Clues: "no email", "text only", "don't send" -> NO EMAIL.                                         │
│             - Clues: "send to x@x.com", "email me" -> YES EMAIL.                                                │
│                                                                                                                 │
│          2. DECISION:                                                                                           │
│             - IF NO EMAIL: Terminate the task immediately by returning "Email skipped by request". DO NOT       │
│  delegate to the Publisher.                                                                                     │
│             - IF YES EMAIL: Delegate the task to the 'Newsletter Publisher' to send the email using the         │
│  content from the Writer.                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

[CrewAIEventsBus] Sync handler error in on_llm_call_started: Only one live display may be active at once

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chief Editor                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Sent                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d0398dac-c11e-4a47-bf17-d791c48a8a9a                                                                     │
│  Agent: Chief Editor                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Sent

## **Lab Work: Build Your Own Agent!**

Now it's your turn to showcase your idea of an application of AI Agent(s).

Instruction:


*   You can form a group or go solo
*   Try to get at least ONE agent with a tool
*   You can use AI to generate code ( Gemini 3 recommended )
*   If you don't want to use complex tool, try to make a mock tool instead ( For example, instead of sending email, you just print out a mock email format)
*   No marking rubric or competition. What matters is your idea, not your execution
*   Don't be shy to ask for help!

